In [26]:
"""LIBRERIAS"""

import h5py
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sn

# Importar dataset

### Por color

In [9]:
""" leer archivos de caracteristicas de color y etiquetas"""
h5f_color   = h5py.File("/home/miguel/Documents/ProyectoMoniliaCacao/2BDD_Caracteristicas/color_caracteristicas.h5", 'r')
h5f_colorEtiq = h5py.File("/home/miguel/Documents/ProyectoMoniliaCacao/2BDD_Caracteristicas/color_etiquetas.h5", 'r')

"""crear cadena de almacenamiento de las caracteristicas de color y de etiquetas"""
color_caracteristicas_str = h5f_color['Cacao_Color_Caracteristicas']
etiquetas_caract_color_str = h5f_colorEtiq['Cacao_Color_Etiquetas']

"""transformar caracteres caracteristicas y etiquetas a vectores"""
caracteristicas_color = np.array(color_caracteristicas_str)
caracteristicas_col_etiquetas  = np.array(etiquetas_caract_color_str)

h5f_color.close()
h5f_colorEtiq.close()

### Por textura

In [10]:
""" leer archivos de caracteristicas de textura con respectivas etiquetas"""
h5f_textura  = h5py.File("/home/miguel/Documents/ProyectoMoniliaCacao/2BDD_Caracteristicas/textura_caracteristicas.h5", 'r')
h5f_textureEtiq = h5py.File("/home/miguel/Documents/ProyectoMoniliaCacao/2BDD_Caracteristicas/textural_etiquetas.h5", 'r')

"""crear cadena de almacenamiento de las caracteristicas de textura con sus etiquetas"""
textura_caracteristicas_str = h5f_textura['Cacao_Textura_Caracteristicas']
textura_caract_etiquetas_str = h5f_textureEtiq['Cacao_Textura_Etiquetas']

caracteristicas_textura = np.array(textura_caracteristicas_str )
caracteristicas_textura_etiqueta  = np.array(textura_caract_etiquetas_str)

h5f_textura.close()
h5f_textureEtiq.close()

### Concatenar caracteristicas

In [18]:
"""unificar las caracteristicas de color y textura"""
global_caracteristica = np.hstack([caracteristicas_color, caracteristicas_textura])
"""mostrar forma del vector caracteristica global"""
print("[STATUS] features shape: {}".format(global_caracteristica.shape))
"""forma de conjunto de etiquetas de los datos obtenidos"""
print("[STATUS] labels shape: {}".format(caracteristicas_textura_etiqueta.shape))

[STATUS] features shape: (1206, 525)
[STATUS] labels shape: (1206,)


## Identificacion de variables y division de datos entrenamiento y prueba

In [20]:
"""variable independiente"""
X = global_caracteristica
"""variable objetivo"""
Y = caracteristicas_col_etiquetas

"""divsion de de datos para entrenar el modelo"""
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                        test_size   = 0.20,
                                        random_state = 9)
"""mostrar dimension de datos entrenamiento"""
X_train.shape, X_test.shape

((964, 525), (242, 525))

# Modelo

In [21]:
"""establecer parametros del kernel sigmoid para clasificacion binaria"""
sigmoid_param_grid = {
        "kernel": ["sigmoid"],
        "gamma": ["auto", "scale"],
        "C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000],
        "decision_function_shape": ['ovo', 'ovr']
}

In [22]:
"""establecer hiperparametros del kernel sigmoid"""
sigmoid_grid = GridSearchCV(svm.SVC(), sigmoid_param_grid)
"""entrenar modelo svm de kernel sigmoid"""
sigmoid_grid.fit(X_train,y_train)
"""mostrar parametros del modelo """
print(sigmoid_grid.best_estimator_)

SVC(C=1000, decision_function_shape='ovo', gamma='auto', kernel='sigmoid')


In [23]:
"""evaluar el modelo"""
sigmoid_pred = sigmoid_grid.predict(X_test)

### Evaluacion del modelo

In [24]:
"""calcular accuracy, precision, recall, y f1score"""
sigmoid_accuracy = accuracy_score(y_test, sigmoid_pred)
sigmoid_precision = precision_score(y_test, sigmoid_pred, average='weighted')
sigmoid_recall = recall_score(y_test, sigmoid_pred, average='weighted')
sigmoid_f1 = f1_score(y_test, sigmoid_pred, average='weighted')

print('Accuracy (Sigmoid Kernel): ', "%.2f" % (sigmoid_accuracy*100))
print('F1 (Sigmoid Kernel): ', "%.2f" % (sigmoid_f1*100))
print('Precision (Sigmoid Kernel): ', "%.2f" % (sigmoid_precision*100))
print('Recall (Sigmoid Kernel): ', "%.2f" % (sigmoid_recall*100))

Accuracy (Sigmoid Kernel):  86.36
F1 (Sigmoid Kernel):  86.41
Precision (Sigmoid Kernel):  86.48
Recall (Sigmoid Kernel):  86.36


In [27]:
"""reposte de metricas de evaluacion estandar """
print(classification_report(y_test, sigmoid_pred))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89       151
           1       0.81      0.84      0.82        91

    accuracy                           0.86       242
   macro avg       0.85      0.86      0.86       242
weighted avg       0.86      0.86      0.86       242

